### Wykrywanie phishingowych stron internetowych - Ekstrakcja cech

Notebook stworzony na potrzebę realizacji pracy magisterskiej pod kierunkiem dr hab. Marka Góździa.

## 1. Cel

Strony phishingowe to powszechna metoda inżynierii społecznej, polegająca na podszywaniu się pod zaufane strony internetowe w celu wyłudzenia danych. Celem tego projektu jest zgromadzenie danych i wyodrębnianie wybranego zestawu cech z adresów URL (z ang. Uniform Resource Locator), na bazie których będzie można spróbować sklasyfikować witrynę jako prawdziwą lub phishingową.

### 2. Gromadzenie danych

Aby osiągnąć zadowalający wynik, potrzebujemy wielu adresów URL zarówno tych prawdziwych (oznaczonych w notebooku jako 0) jak i phishignowych (oznaczonych w notebooku jako 1 oraz "podejrzana").

Do zebrania adresów phishingowych wykorzystamy trzy różne serwisy, udostępniające i agregujące tego rodzaju dane.

1. OpenPhish - OpenPhish jest popularnym serwisem, który udostępnia listę zidentyfikowanych stron phishingowych. Miejscem, z którego pobierzemy dane będzie plik https://openphish.com/feed.txt.

2. PhishTank - Ta usługa udostępnia zestaw adresów phishingowych w wielu formatach, takich jak CSV, JSON itp., które są aktualizowane co godzinę. Aby pobrać dane skorzystamy z udostępnionych plików na stronie https://www.phishtank.com/developer_info.php

3. Urlscan.io - Serwis ten pozwala na skanowanie i analizę URLi, w tym wykrywanie stron phishingowych. Oferuje również API, które można wykorzystać do pobierania informacji o prawdziwych, oraz podejrzanych adresach URL. 

W przypadku adresów URL prawdziwych (bezpiecznych) stron, wykorzystamy listę adresów URL oficjalnych stron, należących do znanych firm, które udostępnia serwis urlscan.io (https://urlscan.io/api/v1/pro/availableBrands), oraz listę najpopularniejszych domen, na podstawie liczby odwołujących się do nich podsieci, udostępnianą przez serwis majestic.com (https://pl.majestic.com/reports/majestic-million)

### 2.1 Adresy phishingowe

### OpenPhish

Adresy phishingowe są pobierane z serwisu OpenPhish poprzez odpowiedni endpoint API. Po pobraniu zestawu danych jest on ładowany do ramki DataFrame openphish_data.

In [2]:
import pandas as pd
import requests

In [13]:
import csv

url = 'https://openphish.com/feed.txt'

response = requests.get(url)
# print(response.text)

if response.status_code == 200:
    content = response.content.decode('utf-8')
    
    urls = content.splitlines()
    
    csv_file_name = 'phishing_urls.csv'
    
    with open(csv_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        
        writer.writerow(['URL'])
        
        for url in urls:
            writer.writerow([url])
    
    print(f'Successfully written to {csv_file_name}')
    
else:
    print('Failed to download the content')

Successfully written to phishing_urls.csv


In [2]:
openphish_data = pd.read_csv('../data/phishing_urls.csv')
openphish_data.head()

,URL
0,https://doiw0zuhcw9rosjhic.pages.dev/smart89/
1,https://ipfs.io/ipfs/bafkreiahpgxlkqme2bhj5jfi...
2,https://cloudflare-ipfs.com/ipfs/bafkreiahpgxl...
3,https://start-a-complaint-meta-01061657.pages....
4,https://distritonorte.com.mx/js/config/login.php/


In [3]:
openphish_data.shape

(500, 1)

### PHISHTANK

Adresy phishingowe są pobierane z PhishTanka poprzez odopowiedni endpoint API. Po pobraniu zestawu danych jest on ładowany do ramki DataFrame phishtank_data.

In [8]:
import gzip
import shutil
import os

url = "http://data.phishtank.com/data/online-valid.csv.gz"

downloaded_file = "online-valid.csv.gz"

response = requests.get(url, stream=True)
with open(downloaded_file, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

with gzip.open(downloaded_file, 'rb') as f_in:
    with open(downloaded_file[:-3], 'wb') as f_out:  # Usunięcie rozszerzenia .gz
        shutil.copyfileobj(f_in, f_out)

os.remove(downloaded_file)

In [4]:
#loading the phishing URLs data to dataframe
phishtank_data = pd.read_csv("../data/online-valid.csv")
phishtank_data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8654566,https://us17.list-manage.com/survey?u=95c29488...,http://www.phishtank.com/phish_detail.php?phis...,2024-07-14T09:28:56+00:00,yes,2024-07-14T09:33:16+00:00,yes,Other
1,8654565,https://newspirit.my/ndanedest/error.php,http://www.phishtank.com/phish_detail.php?phis...,2024-07-14T09:23:59+00:00,yes,2024-07-14T09:33:16+00:00,yes,Other
2,8654563,https://africantrailphotography.co.za/invsecwm...,http://www.phishtank.com/phish_detail.php?phis...,2024-07-14T09:23:16+00:00,yes,2024-07-14T09:33:16+00:00,yes,Other
3,8654561,https://btinternetwebmailogin.weeblysite.com/,http://www.phishtank.com/phish_detail.php?phis...,2024-07-14T09:22:03+00:00,yes,2024-07-14T09:33:16+00:00,yes,Other
4,8654559,http://2p352yrrr.duckdns.org,http://www.phishtank.com/phish_detail.php?phis...,2024-07-14T09:20:30+00:00,yes,2024-07-14T09:33:16+00:00,yes,Other


In [5]:
phishtank_data.shape

(62281, 8)

Pobrany zbiór danych z PhishTanka zawiera 62281 wierszy. W celu zredukowania ilości danych, wybierzemy losowo 4,000 adresów URL. Wybrane adresy zapiszemy ramki DataFrame selected_phishtank.

In [6]:
#Collecting 4,000 Phishing URLs randomly
selected_phishtank = phishtank_data.sample(n = 4000, random_state = 12).copy()
selected_phishtank = selected_phishtank.reset_index(drop=True)
selected_phishtank.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8585872,https://allegrolokalnie.c8a7d9a7.pl/oferta/873...,http://www.phishtank.com/phish_detail.php?phis...,2024-05-24T08:09:03+00:00,yes,2024-05-28T07:33:34+00:00,yes,Allegro
1,8625776,https://www.trx016.asia/a/,http://www.phishtank.com/phish_detail.php?phis...,2024-06-23T11:19:42+00:00,yes,2024-06-23T12:04:35+00:00,yes,Other
2,8229192,https://cloudflare-ipfs.com/ipfs/bafybeihdtw7y...,http://www.phishtank.com/phish_detail.php?phis...,2023-07-19T00:40:39+00:00,yes,2023-07-19T00:43:00+00:00,yes,Other
3,8565299,https://docs.google.com/presentation/d/e/2PACX...,http://www.phishtank.com/phish_detail.php?phis...,2024-05-06T18:02:50+00:00,yes,2024-05-06T18:13:04+00:00,yes,Other
4,8633157,https://oolx.2384237.xyz/captcha,http://www.phishtank.com/phish_detail.php?phis...,2024-06-28T18:40:11+00:00,yes,2024-06-28T18:43:18+00:00,yes,Other


In [7]:
selected_phishtank.shape

(4000, 8)

### URLSCAN

Adresy phishingowe są wczytywane z wygenerowanego pliku urlscanpro-export.csv. Plik ten zawiera informacje o adresach URL sklasyfikowanych jako phishingi. Po pobraniu zestawu danych jest on ładowany do ramki DataFrame urlscan_data.

In [8]:
urlscan_data = pd.read_csv('../data/urlscanpro-export.csv', sep=';')
urlscan_data.head()

,Scan Date,Scan Method,Scan Source,Visibility,Submitter,Scan URL,Task URL,Task Domain,Task Apex Domain,Task Tags,...,Page PTR Record,Page ASN,Page AS Name,Page Server,Page URL,Page Domain,Page Apex Domain,Malicious,Brands,Labels
0,2024-07-14T11:48:25.694Z,api,NaN,unlisted,us,https://urlscan.io/result/2cc27262-c12f-46c1-a...,https://yeed9.z35.web.core.windows.net/windows...,yeed9.z35.web.core.windows.net,windows.net,NaN,...,NaN,AS8075,"MICROSOFT-CORP-MSN-AS-BLOCK, US",Windows-Azure-Web/1.0 Microsoft-HTTPAPI/2.0,https://yeed9.z35.web.core.windows.net/windows...,yeed9.z35.web.core.windows.net,windows.net,True,techsupportscam,NaN
1,2024-07-14T11:48:11.973Z,automatic,urlscan-observe,unlisted,NaN,https://urlscan.io/result/7ae49603-bf43-42a5-9...,https://ausreportinbox.com/,ausreportinbox.com,ausreportinbox.com,NaN,...,NaN,AS198953,"PROTON66, RU",openresty/1.25.3.1,https://ausreportinbox.com/,ausreportinbox.com,ausreportinbox.com,True,govau,NaN
2,2024-07-14T11:48:10.785Z,automatic,urlscan-observe,unlisted,NaN,https://urlscan.io/result/2fd897c3-3940-4144-b...,https://iade-temmuz.sbs/,iade-temmuz.sbs,iade-temmuz.sbs,NaN,...,cpanel16.offshoreracks.com,AS52469,"Offshore Racks S.A, PA",Apache,https://iade-temmuz.sbs/,iade-temmuz.sbs,iade-temmuz.sbs,True,bddk,NaN
3,2024-07-14T11:46:42.992Z,api,NaN,public,us,https://urlscan.io/result/6009a640-5c0e-4d11-8...,https://srz.atazanavir.org/,srz.atazanavir.org,atazanavir.org,NaN,...,server.geekwebserver.com,AS393960,"HOST4GEEKS-LLC, US",Apache,https://srz.atazanavir.org/,srz.atazanavir.org,atazanavir.org,True,salesforce,NaN
4,2024-07-14T11:46:32.622Z,api,NaN,public,fi,https://urlscan.io/result/065e550e-999d-48bd-8...,https://shestripping.com/wp-content/plugins/ar...,shestripping.com,shestripping.com,@phish_report,...,ec2-54-235-160-238.compute-1.amazonaws.com,AS14618,"AMAZON-AES, US",Apache/2.4.39 (Amazon) PHP/5.6.33,https://shestripping.com/wp-content/plugins/ar...,shestripping.com,shestripping.com,True,nedbank,content.login


In [9]:
urlscan_data.shape

(11896, 22)

Zbiór danych z urlscan.io zawiera 11896 wierszy. W celu zredukowania ilości danych, wybierzemy losowo 1,500 adresów URL. Wybrane w ten sposób adresy zapiszemy ramki DataFrame selected_urlscan.

In [10]:
selected_urlscan = urlscan_data.sample(n = 1500, random_state = 12).copy()
selected_urlscan = selected_urlscan.reset_index(drop=True)
selected_urlscan.head()

,Scan Date,Scan Method,Scan Source,Visibility,Submitter,Scan URL,Task URL,Task Domain,Task Apex Domain,Task Tags,...,Page PTR Record,Page ASN,Page AS Name,Page Server,Page URL,Page Domain,Page Apex Domain,Malicious,Brands,Labels
0,2024-07-14T02:23:29.726Z,automatic,openphish,public,NaN,https://urlscan.io/result/f2d0a3d0-df02-4fd9-a...,http://reviews.bussinesaccount.my.id/rever.php,reviews.bussinesaccount.my.id,bussinesaccount.my.id,NaN,...,NaN,AS14061,"DIGITALOCEAN-ASN, US",Apache,https://reviews.bussinesaccount.my.id/rever.php,reviews.bussinesaccount.my.id,bussinesaccount.my.id,True,facebook,NaN
1,2024-07-14T02:14:18.387Z,automatic,openphish,public,NaN,https://urlscan.io/result/a90a7fec-62a0-492d-a...,https://amazon-login-nine.vercel.app/,amazon-login-nine.vercel.app,amazon-login-nine.vercel.app,NaN,...,NaN,AS16509,"AMAZON-02, US",Vercel,https://amazon-login-nine.vercel.app/,amazon-login-nine.vercel.app,amazon-login-nine.vercel.app,True,amazon,hosting.free
2,2024-07-13T23:38:51.389Z,api,NaN,unlisted,us,https://urlscan.io/result/057096b9-6a17-4510-b...,http://ssxcs.com/si6bln/?t=eqav2h0gta/index/us...,ssxcs.com,ssxcs.com,NaN,...,NaN,AS55933,"CLOUDIE-AS-AP Cloudie Limited, HK",nginx,https://ssxcs.com/index/user/login.html,ssxcs.com,ssxcs.com,True,genericemail,NaN
3,2024-07-14T08:38:18.570Z,api,NaN,public,us,https://urlscan.io/result/c910185d-427e-4b2b-a...,https://qcc.atazanavir.org/,qcc.atazanavir.org,atazanavir.org,NaN,...,server.geekwebserver.com,AS393960,"HOST4GEEKS-LLC, US",Apache,https://qcc.atazanavir.org/,qcc.atazanavir.org,atazanavir.org,True,salesforce,NaN
4,2024-07-13T23:10:43.898Z,api,NaN,public,us,https://urlscan.io/result/7efebfbe-3c66-46a4-8...,http://pub-8e34ba5d87084c3f9c5d2a23075f50b3.r2...,pub-8e34ba5d87084c3f9c5d2a23075f50b3.r2.dev,pub-8e34ba5d87084c3f9c5d2a23075f50b3.r2.dev,NaN,...,NaN,AS13335,"CLOUDFLARENET, US",cloudflare,https://pub-8e34ba5d87084c3f9c5d2a23075f50b3.r...,pub-8e34ba5d87084c3f9c5d2a23075f50b3.r2.dev,pub-8e34ba5d87084c3f9c5d2a23075f50b3.r2.dev,True,genericemail,"hosting.static,software.cpanel,hosting.cdn"


In [11]:
selected_urlscan.shape

(1500, 22)

Zebrane adresy phishingowe z tych trzech źródeł złączymy w jedną ramkę danych phishurl, a następnie zapiszemy do pliku phishurl.csv.

In [12]:
# Select only the URL column from each data frame
openphish_urls = openphish_data[['URL']].rename(columns={'URL': 'url'})
selected_phishtank_urls = selected_phishtank[['url']]
selected_urlscan_urls = selected_urlscan[['Page URL']].rename(columns={'Page URL': 'url'})

# Combine all data frames into one
phishurl = pd.concat([openphish_urls, selected_phishtank_urls, selected_urlscan_urls], ignore_index=True)

# Display the first few lines of the resulting data frame
phishurl.head()

,url
0,https://doiw0zuhcw9rosjhic.pages.dev/smart89/
1,https://ipfs.io/ipfs/bafkreiahpgxlkqme2bhj5jfi...
2,https://cloudflare-ipfs.com/ipfs/bafkreiahpgxl...
3,https://start-a-complaint-meta-01061657.pages....
4,https://distritonorte.com.mx/js/config/login.php/


Połączone dane phishingowe zawierają 6,000 adresów URL. Dane te nie zawierają dupliaktów.

In [13]:
phishurl.shape

(6000, 1)

Zebrane adresy phishingowe zapiszemy do pliku csv phishurl.csv.

In [14]:
# Storing the phishing URLs data to csv file
phishurl.to_csv('../data/phishurl.csv', index=False)

### 2.2. Prawidłowe (Bezpieczne) adresy URL

### URLSCAN

Adresy bezpiecznych stron z serwisu urlscan.io są wczytywane z wygenerowanego pliku URLSCAN_legit_domains.csv. Po pobraniu zestawu danych jest on ładowany do ramki DataFrame legit_domains.urlscan.

In [5]:
legit_domains_urlscan = pd.read_csv('../data/URLSCAN_legit_domains.csv', names=['url'], header=0)
legit_domains_urlscan.head()

,url
0,https://dewatergroep.be
1,https://poppankki.fi
2,https://ceskaposta.cz
3,https://suncoastcreditunion.com
4,https://caf.fr


Zbiór bezpiecznych adresów URL z urlscan.io zawiera 1288 wierszy.

In [16]:
legit_domains_urlscan.shape

(1288, 1)

### MAJESTIC

Adresy prawdziwych stron z serwisu majestic.com są wczytywane z wygenerowanego pliku majestic_milion.csv. Po pobraniu zestawu danych jest on ładowany do ramki DataFrame. W celu zredukowania ilości danych, wybierzemy pierwsze 4933 wiersze, które następnie zapiszemy do ramki danych legit_domain_majestic.

In [17]:
majestic_milion = pd.read_csv('../data/majestic_million.csv')

legit_domains_majestic = majestic_milion.iloc[:4933]
legit_domains_majestic.head()

,GlobalRank,TldRank,Domain,TLD,RefSubNets,RefIPs,IDN_Domain,IDN_TLD,PrevGlobalRank,PrevTldRank,PrevRefSubNets,PrevRefIPs,Url
0,1,1,google.com,com,473803,2145427,google.com,com,1,1,472906,2138041,https://google.com
1,2,2,facebook.com,com,465522,2224144,facebook.com,com,2,2,464974,2220575,https://facebook.com
2,3,3,youtube.com,com,422420,1844676,youtube.com,com,3,3,421746,1842025,https://youtube.com
3,4,4,twitter.com,com,402407,1715216,twitter.com,com,4,4,401716,1713711,https://twitter.com
4,5,5,instagram.com,com,367005,1597839,instagram.com,com,5,5,366554,1595937,https://instagram.com


In [18]:
legit_domains_majestic.shape

(4933, 13)

Zebrane adresy prawdziwych stron z serwisów urlscan.io i majestic.com złączymy w jedną ramkę danych legiurl, a następnie zapiszemy do pliku legiurl.csv.

In [19]:
legit_domains_majestic = legit_domains_majestic[['Url']].rename(columns={'Url': 'url'})
legiurl = pd.concat([legit_domains_urlscan, legit_domains_majestic], ignore_index=True)

legiurl.head()

,url
0,https://dewatergroep.be
1,https://poppankki.fi
2,https://ceskaposta.cz
3,https://suncoastcreditunion.com
4,https://caf.fr


In [20]:
legiurl.shape

(6221, 1)

Zebraliśmy 6221 adresów prawdziwych stron internetowych, z czego dane te mogą zawierać duplikaty, ponieważ łączone są z dwóch różnych źródeł. W celu usunięcia duplikatów, wykonujemy operację drop_duplicates. 

In [22]:
legiurl.drop_duplicates(inplace=True)
legiurl.shape

(6000, 1)

W ten sposób uzyskaliśmy 6000 unikalnych adresów URL bezpiecznych stron internetowych. Zapisujemy je do pliku legiurl.csv.

In [131]:
# Storing the legitimate URLs data to csv file
legiurl.to_csv('../data/legiurl.csv', index=False)

### 3. Ekstrakcja cech

Na tym etapie są wyodrębniane poszczególne, wybrane cechy ze zbioru adresów URL oraz stron internetowych, do których prowadzą.

Funkcje w tej sekcji zostały podzielone na trzy kategorie

1. Funkcje oparte na pasku adresu (Adres URL)
2. Funkcje oparte na cechach domeny
3. Funkcje oparte na kodzie HTML i JavaScript stron internetowych

### 3.1. Funkcje oparte na adresie URL:
Z adresu URL można wyodrębnić wiele informacji, które mogą zostać poźniej poddane dalszej analizie. Część z nich może świadczyć o tym, że strona powinna być uznana za podejrzaną. Poniższe metody wyodrębnienia niektórych informacji z adresu URL zostały zaimplementowane w tym projekcie.

1. Domena w adresie URL
2. Adres IP w adresie URL
3. Symbol „@” w adresie URL
4. Długość adresu URL
5. Poziom zagnieżdżenia adresu URL
6. Przekierowanie „//” w adresie URL
7. „http/https” w nazwie domeny
8. Korzystanie z usług skracania adresów URL
9. Prefiks lub sufiks „-” w domenie

Każda z tych funkcji została wyjaśniona i zakodowana poniżej:

In [23]:
from urllib.parse import urlparse
import ipaddress

### 3.1.1. Domena w adresie URL
Tutaj wyodrębniamy domenę obecną w adresie URL. Ta funkcja nie ma większego znaczenia w treningu, dlatego zostanie zignorowana podczas trenowania modelu.

In [24]:
# 1.Domain of the URL (Domain) 
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.", domain):
        domain = domain.replace("www.", "")
    return domain

### 3.1.2. Adres IP w adresie URL
Funkcja sprawdza obecność adresu IP w adresie URL. Adresy URL mogą zawierać adres IP zamiast nazwy domeny, jednak nie jest to typowe dla bezpiecznych stron internetowych. Jeśli zamiast nazwy domeny w adresie URL zostanie użyty adres IP, powinniśmy założyć, że strona jest podejrzana.

Jeśli w adresie URL występuje adres IP, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [25]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
    try:
        if url.startswith('http://') or url.startswith('https://'):
            url = re.sub(r'https?://', '', url)
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip

### 3.1.3. Symbol „@” w adresie URL
Funkcja sprawdza obecność symbolu „@” w adresie URL. Użycie symbolu „@” w adresie URL powoduje, że przeglądarka ignoruje wszystko poprzedzające symbol „@”, a prawdziwy adres często następuje po symbolu „@”.

Jeśli adres URL zawiera symbol „@”, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [26]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
    if "@" in url:
        at = 1
    else:
        at = 0
    return at

### 3.1.4. Długość adresu URL
Funkcja oblicza długość adresu URL. Oszuści mogą użyć długiego adresu URL, aby ukryć podejrzaną część w pasku adresu. W tym projekcie zakładamy, że jeśli długość adresu URL jest większa lub równa 54 znaków, wówczas adres URL jest klasyfikowany jako phishing, w przeciwnym razie uznajemy adres URL za prawdziwy.

Jeśli długość całego adresu URL >= 54, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [27]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
    if len(url) < 54:
        length = 0
    else:
        length = 1
    return length

### 3.1.5. Poziom zagnieżdżenia adresu URL
Ta funkcja oblicza liczbę podstron w adresie URL na podstawie znaku „/”.

Zwracana wartość funkcji jest liczbą.

In [28]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth + 1
    return depth

### 3.1.6. Przekierowanie „//” w adresie URL
Funkcja sprawdza obecność „//” w adresie URL. Obecność znaku „//” w ścieżce adresu URL oznacza, że użytkownik zostanie przekierowany na inną stronę internetową. Jeśli adres URL zaczyna się od „HTTP”, znak „//” powinien pojawić się na szóstej pozycji, jeśli jednak adres URL wykorzystuje protokół „HTTPS”, wówczas „//” powinien pojawić się na siódmej pozycji.

Jeśli znak „//” znajduje się w dowolnym innym miejscu adresu URL poza protokołem, wartość przypisana do tej funkcji wynosi 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [29]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0

### 3.1.7. „https” w nazwie domeny
Funkcja sprawdza obecność nazwy protokołu „https” w części adresu URL dotyczącej domeny. Oszuści mogą dodać „https” do części adresu URL dotyczącej domeny, aby oszukać użytkowników, że strona jest szyfrowana. Jeśli adres URL zawiera „https” w części domeny, może to być próba oszustwa, ponieważ protokół szyfrowania powinien znajdować się tylko na początku adresu URL, a nie w jego domenie. Taka taktyka może być wykorzystywana przez oszustów do stworzenia fałszywego wrażenia bezpieczeństwa strony. 

Jeśli adres URL zawiera „http/https” w części domeny, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [30]:
# 7.Existence of “HTTPS” in the Domain Part of the URL (https_Domain)
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 1
    else:
        return 0

### 3.1.8. Korzystanie z usług skracania adresów URL „TinyURL”
Skracanie adresu URL to metoda zmniejszania liczby znaków w linku, w której adres URL może być znacznie krótszy, a mimo to prowadzić do żądanej strony internetowej. Odbywa się to poprzez przekierowanie HTTP w przypadku krótkiej nazwy domeny, która prowadzi do docelowej strony internetowej.

Jeśli adres URL korzysta z popularnych usług skracania, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [31]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net|bl\.ink|rebrandly|clickmeter|cuttly|sniply|hyperlink|dub|zapier|short\.io"

In [32]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services, url)
    if match:
        return 1
    else:
        return 0

### 3.1.9. Prefiks lub sufiks „-” w domenie
Sprawdzanie obecności „-” w części adresu URL dotyczącej domeny. Symbol myślnika jest rzadko używany w prawidłowych adresach URL. Oszuści często używają przedrostków lub sufiksów oddzielonych (-) w nazwie domeny, aby użytkownicy mieli wrażenie, że mają do czynienia z legalną stroną internetową (np. umcs-edu.pl).

Jeśli adres URL zawiera symbol „-” w części adresu URL dotyczącej domeny, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [33]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1           
    else:
        return 0            

### 3.2. Funkcje oparte na domenie:
Pobieranie poniższych informacji zostało uwzględnione w tym projekcie.

1. Rekord DNS
2. Ruch w witrynie
3. Wiek domeny
4. Czas od utworzenia domeny

Każda z tych funkcji została wyjaśniona i zakodowana poniżej:

In [99]:
!pip install python-whois


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import re
from bs4 import BeautifulSoup
import whois
from datetime import datetime

### 3.2.1. Rekord DNS
W przypadku witryn phishingowych strona często nie jest jeszcze rozpoznawana przez bazę danych WHOIS, albo nie istnieją żadne informacje dotyczące jej nazwy. Jeśli rekord DNS jest pusty lub nie został znaleziony, wartość przypisana do tej funkcji wynosi 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

Ekstrakcja tej informacji została zaimplementowana w funkcji featureExtraction.

In [ ]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function

### 3.2.2. Wiek domeny
Tą informację można wyodrębnić z bazy danych WHOIS. Większość witryn phishingowych działa przez krótki czas. W przypadku tego projektu minimalny okres legalnej domeny założymy jako 12 miesięcy (ponieważ na taki okres zwykle rejestrowane są domeny phishingowe). Wiek jest liczony jako różnica pomiędzy czasem utworzenia i wygaśnięcia.

Jeśli okres domeny < 12 miesięcy, wartość tej funkcji wynosi 1 (strona podejrzana), w przeciwnym razie 0 (strona prawdziwa).

In [35]:
# 13.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_info):
    try:
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        if not isinstance(creation_date, datetime):
            try:
                creation_date = datetime.strptime(creation_date, "%Y-%m-%d")
            except TypeError:
                return 1  # Unable to parse date
            except ValueError:
                # Handle cases like "Before 2001"
                if "before" in creation_date.lower():
                    return 1  # Treat as old domain
                else:
                    return 1  # Unable to parse date

        today = datetime.now()
        age = today.year - creation_date.year - ((today.month, today.day) < (creation_date.month, creation_date.day))
        return age
    except AttributeError:
        return 1  # Domain information not available

### 3.2.3. Czas od utworzenia domeny
Funkcja oblicza liczbę dni, która upłynęła od daty rejestracji domeny do chwili obecnej (momentu wykonania programu). Jeśli ten okres jest krótszy niż 3 miesiące (90 dni), funkcja zwraca 1 (strona podejrzana), w przeciwnym razie zwraca 0 (strona prawdziwa).

In [36]:
# 14.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_info):
    if domain_info is None:
        return 1  # Unable to retrieve domain info

    creation_date = domain_info.creation_date

    if creation_date is None:
        return 1  # Missing dates / No information

    # Ensure creation_date is a datetime object
    if isinstance(creation_date, list):
        creation_date = creation_date[0]
    if not isinstance(creation_date, datetime):
        try:
            creation_date = datetime.strptime(creation_date, "%Y-%m-%d")
        except (TypeError, ValueError):
            return 1  # Unable to parse date

    days_since_creation = (datetime.now() - creation_date).days

    return 1 if (days_since_creation < 90) else 0


### 3.3. Funkcje oparte na HTML i JavaScript
Pobranie informacji o poniższych zdarzeniach zostało uwzględnione w tym projekcie.

1. Przekierowanie ramki IFrame
2. Dostosowywanie paska stanu
3. Wyłączanie kliknięcia prawym przyciskiem myszy
4. Przekazywanie strony internetowej

Każda z tych funkcji została wyjaśniona i zakodowana poniżej:

In [37]:
# importing required packages for this section
import requests

### 3.3.1. Przekierowanie ramki IFrame
IFrame to znacznik HTML używany do wyświetlania dodatkowej strony internetowej w aktualnie odwiedzanej witrynie. Oszuści mogą wykorzystać tag „iframe” i uczynić go niewidocznym, tj. pozbawionym obramowań ramki. W tym celu mogą wykorzystać różne atrybuty.

Jeśli ramka iframe jest pusta lub nie zostanie znaleziona odpowiedź, wartość przypisana do tej funkcji to -1, jeśli wykryto podejrzany atrybut 1 (strona podejrzana) lub 0 (strona prawdziwa) w pozostałym przypadku.

In [38]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
    if not response or not hasattr(response, 'text'):
        return -1
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    iframes = soup.find_all('iframe')
    
    if not iframes:
        return 0
    
    for iframe in iframes:
        if 'style' in iframe.attrs and 'display:none' in iframe['style']:
            return 1
        if 'width' in iframe.attrs and iframe['width'] == '0':
            return 1
        if 'height' in iframe.attrs and iframe['height'] == '0':
            return 1
        if 'frameborder' in iframe.attrs and iframe['frameborder'] == '0':
            return 1
    return 0

### 3.3.2. Dostosowywanie paska stanu
Oszuści mogą używać JavaScriptu do wyświetlania użytkownikom fałszywego adresu URL na pasku stanu. Aby wyodrębnić tę funkcję, musimy wydobyć kod źródłowy strony internetowej, a następnie zdarzenie „onMouseOver” i sprawdzić, czy wprowadza jakieś zmiany na pasku stanu.

Jeśli odpowiedź jest pusta przypisana wartość to -1, jeśli zostanie znaleziona funkcja onmouseover, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [39]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response):
    if not response or not hasattr(response, 'text'):
        return -1
    
    if re.findall(r'onmouseover', response.text, re.IGNORECASE):
        return 1
    else:
        return 0

### 3.3.3. Wyłączanie kliknięcia prawym przyciskiem myszy
Oszuści wykorzystują JavaScript do wyłączania funkcji kliknięcia prawym przyciskiem myszy, dzięki czemu użytkownicy nie mogą przeglądać i zapisywać kodu źródłowego strony internetowej. Dla tej funkcji będziemy szukać zdarzenia „event.button==2” w kodzie źródłowym strony i sprawdzać, czy użycie prawego przycisku myszy nie zostało wyłączone.

Jeśli odpowiedź jest pusta, zwracana wartość fukcji to -1, jeśli zostanie znalezione polecenie onmouseover, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [40]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
    if not response or not hasattr(response, 'text'):
        return -1
    
    if re.findall(r"event.button ?== ?2", response.text):
        return 1
    else:
        return 0

### 3.3.4. Przekierowanie strony internetowej
Podejrzane strony internetowe często robią kilka przekierowań, zanim użytkownik otrzyma stronę docelową. Złóżmy, że legalne strony internetowe zostały przekierowane maksymalnie dwa razy. Jeśli liczba przekierowań jest większa niż 2, wartość przypisana do tej funkcji to 1 (strona podejrzana) lub 0 (strona prawdziwa) w przeciwnym wypadku.

In [41]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
    if not response or not hasattr(response, 'history'):
        return -1
    
    if len(response.history) <= 2:
        return 0
    else:
        return 1

### 4. Obliczanie wszystkich funkcji
Poniżej znajduje się zbiorcza metoda, która wywołuje inne funkcje i przechowuje wszystkie pobrane wyniki na liście. Wszystkim przetwarzanym adresom zostanie przypisana etykieta informująca o tym, czy dany adres URL jest phishighiem, czy też nie. Z pomocą tej funkcji wyodrębnimy cechy zarówno stron prawdziwych, jak i phishingowych.

In [42]:
#Function to extract features
def featureExtraction(url, label):
    features = []
    # Address bar based features (9)
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(prefixSuffix(url))

    # Domain based features (3)
    dns = 0
    try:
        domain_info = whois.whois(urlparse(url).netloc)
    except:
        dns = 1
    
    features.append(dns)
    features.append(1 if dns == 1 else domainAge(domain_info))
    features.append(1 if dns == 1 else domainEnd(domain_info))

    # HTML & Javascript based features (4)
    try:
        response = requests.get(url, timeout=10)
    except:
        response = ""

    features.append(iframe(response))
    features.append(mouseOver(response))
    features.append(rightClick(response))
    features.append(forwarding(response))
    features.append(label)

    return features

### 4.1. Bezpieczne adresy URL:
Teraz wyodrębnianie funkcji wykonamy na adresach URL prawdziwych, bezpiecznych stron internetowych.

In [43]:
legiurl.shape

(6000, 1)

In [44]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(legiurl.shape[0]):
   try:
       url = legiurl['url'][i]
       legi_features.append(featureExtraction(url,label))
       print(i, url)
   except Exception as e:
       print(f"Error processing URL at index {i}: {e}")
       with open('log.txt', 'a') as f:
           f.write(f"Error processing URL at index {i}: {e}\n")
       continue

0 https://dewatergroep.be
1 https://poppankki.fi
2 https://ceskaposta.cz
3 https://suncoastcreditunion.com
4 https://caf.fr
5 https://teliacompany.com
6 https://ionos.com
7 https://bancaditalia.it
8 https://gazprom.com
9 https://paribu.com
10 https://vectrabank.com
11 https://bsm.sm
12 https://rakbank.ae
13 https://free.fr
14 https://bakerhudsonhealth.com
15 https://ems.com.cn


2024-07-15 09:10:58,786 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out


16 https://correo.com.uy
17 https://caisse-epargne.fr
18 https://mps.it
19 https://ac.uk


2024-07-15 09:11:02,717 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


20 https://bpi.com.ph
21 https://kanagawabank.co.jp
22 https://popularenlinea.com
23 https://optimum.net
24 https://bankofireland.com
25 https://redsys.es
26 https://fibank.bg
27 https://mgmresorts.com
28 https://backblaze.com
29 https://mitid.dk


...

5920 https://google.cg
5921 https://tv-asahi.co.jp
5922 https://umb.edu
5923 https://binghamton.edu


2024-07-15 13:20:11,057 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out


5924 https://gub.uy
5925 https://thewaltdisneycompany.com
5926 https://victoriassecret.com
5927 https://questionpro.com
Error processing URL at index 5928: 5928
5929 https://princetonreview.com
5930 https://opb.org
5931 https://jhsph.edu
5932 https://ucc.ie
5933 https://break.com
5934 https://agu.org
5935 https://eetimes.com
5936 https://gnavi.co.jp
5937 https://denofgeek.com
5938 https://nespresso.com
5939 https://tineye.com
5940 https://publicradio.org
5941 https://www.ne.jp
5942 https://jacksonville.com
5943 https://uba.ar
5944 https://veeam.com
5945 https://bepress.com
5946 https://google.ge
5947 https://dreamhosters.com
5948 https://bravenet.com


2024-07-15 13:21:37,977 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out


5949 https://google.dz
5950 https://gazzettaufficiale.it
5951 https://highsnobiety.com
5952 https://wwnorton.com
5953 https://fudan.edu.cn
5954 https://google.cat
5955 https://umaine.edu
5956 https://mercer.com
5957 https://nysed.gov
5958 https://invisionapp.com
5959 https://rlp.de
5960 https://dynadot.com
5961 https://uwyo.edu
5962 https://blubrry.com
5963 https://random.org
5964 https://zenbusiness.com
5965 https://thelocal.se
5966 https://hackster.io
5967 https://fau.edu
5968 https://nbcphiladelphia.com
5969 https://cheatsheet.com
5970 https://bethesda.net
5971 https://google.co.ck
5972 https://kompas.com
5973 https://globalsign.com
5974 https://asahi-net.or.jp
5975 https://muenchen.de
5976 https://ga.us
5977 https://curtin.edu.au
5978 https://couriermail.com.au
5979 https://canoe.ca
5980 https://igi-global.com
5981 https://joelonsoftware.com
5982 https://smartrecruiters.com
5983 https://city.ac.uk
5984 https://stores.jp
5985 https://google.com.ag
5986 https://thisamericanlife.org
5

In [45]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Domain_Age', 
                 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards',
                 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,dewatergroep.be,0,0,0,0,0,0,0,0,0,12,0,1,0,0,0,0
1,poppankki.fi,0,0,0,0,0,0,0,0,0,14,0,0,0,0,0,0
2,ceskaposta.cz,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0
3,suncoastcreditunion.com,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0
4,caf.fr,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0


Części z adresów URL nie udało się przetworzyć, w wyniku czego wyodrębniliśmy cechy 5783 bezpiecznych stron. Informacje o błędach zostały zapisane do pliku log.txt.

In [46]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('legitimate.csv', index= False)

### 4.2. Adresy phishingowe:
Ekstrakcja funkcji zostaje przeprowadzana dla adresów URL stron phishingowych.

In [47]:
phishurl.shape

(6000, 1)

In [48]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(phishurl.shape[0]):
  try:
    url = phishurl['url'][i]
    phish_features.append(featureExtraction(url,label))
    print(i, url)
  except Exception as e:
    print(f"Error processing URL at index {i}: {e}")
    with open('log2.txt', 'a') as f:
        f.write(f"Error processing URL at index {i}: {e}\n")
    continue

0 https://doiw0zuhcw9rosjhic.pages.dev/smart89/
1 https://ipfs.io/ipfs/bafkreiahpgxlkqme2bhj5jfivspo6mfawwcwnlp3hx7ig2432osdya2da4
2 https://cloudflare-ipfs.com/ipfs/bafkreiahpgxlkqme2bhj5jfivspo6mfawwcwnlp3hx7ig2432osdya2da4
3 https://start-a-complaint-meta-01061657.pages.dev/
4 https://distritonorte.com.mx/js/config/login.php/
5 https://pub-dc4276801daa4b3cb0fcdc86b134bf2e.r2.dev/cap1.html


2024-07-15 13:24:48,769 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target machine actively refused it


6 https://trimmer.to/uuhmU
7 http://soodaryan.github.io/spotify-web-player-clone
8 https://homepage--sso-kucoin.webflow.io/
9 https://suitee-trezr-app-apps.webflow.io/
10 https://sso-m-robinhoods----com-autth.webflow.io/
11 https://usapholdlogin-1.gitbook.io/eu
12 https://usupheld-lugni.gitbook.io/us
13 https://mettamask--exxtnson-chrome.gitbook.io/us
14 https://www.metaappealform.com/metahelp/
15 https://3656ee.com:8989/mobile-client/bet365_627/index.html?td_channelid=627
16 https://meittamaxlogiin.gitbook.io/us
17 https://dana-van.danakitakan.my.id/
18 https://meine-commerz-phototan-aktualisierung.top/
19 https://www.instagram.com.sg3.sosis-berurat.works/
20 https://www.instagram.com.sg1.sosis-berurat.works/
21 https://indranilpaul15.github.io/netflix-clone
22 https://mdfaizan-786.github.io/Netflix-Clone
23 https://matamsklogin.gitbook.io/us
24 http://claim-dana-kaget.pages.dev/
25 https://free.aslvpn.loseyourip.com/
26 https://visitor132677.com/
27 https://whatsapp-kf.top/
28 https:

2024-07-15 13:35:32,586 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target machine actively refused it


416 https://rb.gy/ykeeqj
417 https://royal-glade-286f.kdsj.workers.dev/privacy/index.php
418 https://www.xuyvdj.com/
419 https://skype-sg.net/
420 https://zigutayoicbj.orbit321.one/index.php
421 http://cf.hk.1111r.icu/
422 http://claimz-dana-kagetzx4.t-4.my.id/
423 http://suite-en-trezor.github.io/us
424 http://skyaldwin.acrif.my.id/
425 http://netflixacademy.kr/
426 https://ipfs.io/ipfs/bafybeieab2k7k4ryxpjiyubtawe5qqq3kkkqrkvkhsb3qreqbqlimk3bce/index.html/
427 http://microsoft.userid-81d.ws/subscription/yJfnBtCfowt-AV53dwkb-bFjJnW1_Yum9WEw=/XVhduIK4AmMt5YNSpZ59Mzl5iKd_6Hnh
428 http://www.lifeproducti.com/page/4382cf8881be544bba2f0011d2888a712da6a226
429 https://it-corner.co.uk/id
430 https://hadiah.dana.kaget.webappsystem.com/
431 http://telesgrnarn.com/
432 https://www.reviews.bussinesaccount.my.id/
433 https://www.reviews.bussinesaccount.my.id/re.php
434 https://www.reviews.bussinesaccount.my.id/rever.php
435 https://www.reviews.bussinesaccount.my.id/reverse.php
436 https://www.rev

...

5906 https://konferencija2023.e-commerce.ba/
5907 https://trade.santanderusedcars.com/
5908 https://seeproc.com.br/test.txt
5909 https://mitmsklogiuma.gitbook.io/us
5910 https://saidtlarrr.net/
5911 https://miusiabc.vip/
5912 https://fdr5ete.pages.dev/robots.txt
5913 http://www.telegrnal.club/
5914 https://agamak.eu/
5915 https://sycinue-rdc.github.io/ITS-Dashboard/
5916 https://ffkelwssnehdsjd.weebly.com/
5917 https://affluent.africa/private/1420/
5918 https://microsoft-secure-alert-8irlc.ondigitalocean.app/?phone=+1-877-200-9638
5919 https://www.oncloudmonsterturkiye.com/
5920 https://www.digicomp.ch/
5921 https://yenib507.top/?e_devlet=1&utm_medium
5922 https://pub-08315c997de948f2a26541af5e3120f3.r2.dev/auth1.html
5923 https://hokaskooutletsalg.com/hoka-one-one-herre-performance-kort-erm-tops-imperial-bl%C3%A3%C2%A3%C3%A2%C2%A3%C3%A3%C2%A2%C3%A2%C2%A5-norge41kan-p-59.html
5924 https://ps-sguser.icu/1/
5925 https://ccc.hmoob123.workers.dev/
5926 https://yy-liness.xyz/
5927 https://a

2024-07-15 16:27:30,816 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


5931 https://konferencija2023.e-commerce.ba/
5932 https://rwg.atazanavir.org/
5933 https://one-digitalservice.ch/login/login?Location=https://one-digitalservice.ch//
5934 https://pub-01d9210ef25e430e90d68cfaf7c9b3be.r2.dev/allroundmen.html?email=t***.s*****@l*****.si
5935 https://pemulihan.akun.dana.webappsystem.com/
5936 https://pub-46ec560551b44f6387ad30dd36f4f128.r2.dev/index.html
5937 https://login.mijnsnapchat.com.accountsettings.app/
5938 https://ios.yahoks.com/
5939 https://www.truereligionoutlet.de/collections/m%c3%a3%c2%a4dchen-jeans-kinder-c-57_58.html?z2vuzgvyptm=
5940 https://www.bet83008.com/
5941 https://mit-borgereboks.com/
5942 https://sowmyasreeravuru.github.io/Project1/
5943 https://pub-02dd55f48a9541ac81a1e7570171f4c4.r2.dev/ebyy.html
5944 https://techincludes.click/
5945 https://www.kontomaniak.pl/
5946 https://thegoldenratioprojects.in/momeweb/Legislation.html
5947 https://uitmuntendshop.nl/login
5948 https://telstra-107806.weeblysite.com/
5949 https://tg3.pages.de

2024-07-15 16:29:41,474 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


5980 https://konferencija2023.e-commerce.ba/
5981 https://lupisan.com/sigin/language.php?0a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e2340a717109c37f32dd0c0b802e7709e234
5982 https://www.dcshoelisboa.com/collections/sapatos-tenis-skate-c-1_3.html?&gender=1
5983 https://kehidupantuanmuda.com/?join=KOITOTO
5984 https://fnal-tiktokshop-rusd.vip/
5985 https://ipfs.eth.aragon.network/ipfs/bafybeifkh27c2b2hqmm2tte5jp6eq3uincyhmxxaukoespre46zjpexybe/
5986 https://www.mephistoslovenska.sk/mephisto-noah-ko%c5%been%c3%a9-panske-oxfordky-sk6550d-hnede-p-100.html
5987 https://www.stanleyphilippines.com/
5988 https://sign-in-att-100076.weeblysite.com/
5989 https://crystalkitchengranitecom.pages.dev/?9a7ee24f27dbe7f9eb705ab891fc5c88

In [49]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record','Domain_Age', 
                 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 
                 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,doiw0zuhcw9rosjhic.pages.dev,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
1,ipfs.io,0,0,1,2,0,0,0,0,0,10,0,0,0,0,0,1
2,cloudflare-ipfs.com,0,0,1,2,0,0,0,1,0,6,0,0,0,0,0,1
3,start-a-complaint-meta-01061657.pages.dev,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1
4,distritonorte.com.mx,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,1


Zebrane dane po przekonwertowaniu zapiszemy do pliku csv phishing.csv.

In [50]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('phishing.csv', index= False)

### 5. Ostateczny zbiór danych
W powyższej sekcji utworzyliśmy dwie ramki danych agregujące cechy stron prawdziwych i phishingowych. Teraz połączymy je w jedną ramkę danych i wyeksportujemy ją do pliku urldata.csv. Plik ten wykorzystamy na etapie machine learningu przeprowadzonego w osobnym notatniku.

In [51]:
#Concatenating the dataframes into one 
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,dewatergroep.be,0,0,0,0,0,0,0,0,0,12,0,1,0,0,0,0
1,poppankki.fi,0,0,0,0,0,0,0,0,0,14,0,0,0,0,0,0
2,ceskaposta.cz,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0
3,suncoastcreditunion.com,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0
4,caf.fr,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0


In [52]:
urldata.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
11777,www-iroblox.com,0,0,0,2,0,0,1,1,0,0,1,0,0,0,0,1
11778,iade-temmuz.sbs,0,0,0,0,0,0,0,1,0,0,1,-1,-1,-1,-1,1
11779,imtoken-bw.ist,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
11780,bet83008.com,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
11781,kajhbscdquiwbjhd.weeblysite.com,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,1


In [53]:
urldata.shape

(11782, 17)

Utworzony zbiór danych zapisujemy do pliku urldata.csv.

In [54]:
# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)

### 6. Wniosek

Cel tego notebooka został osiągnięty. Wyodrębniliśmy 16 funkcji dla 11782 adresów URL, na które składa się 5999 adresów phishingowych i 5783 bezpiecznych adresów URL.

### 7. Źródła

```
https://archive.ics.uci.edu/ml/datasets/Phishing+Websites
https://pl.majestic.com/reports/majestic-million
https://pro.urlscan.io/search?additionalFilter=brand.key%3A*%3Bdate%3A%3Enow-24h%3Bpage.status%3A%22200%22&collapse=page.apexDomain
https://pro.urlscan.io/search?additionalFilter=brand.key%3A*&summary=true
https://openphish.com/
https://phishtank.org/
```
